# Реализация алгоритма Advantage-Actor Critic (A2C) (вплоть до 10 баллов)

#### дедлайн задания (сразу жёсткий): 26 апреля, 23:59 UTC+3

# Работа выполнена: Терентьев Александр Андреевич, Б05-003.

В данной работе Вам предстоит реализовать алгоритм `Advantage Actor Critic`, обучаемый на батче из сред `Atari 2600`, работающих параллельно.

Для начала будут использованы обёртки сред, реализованные в файле `atari_wrappers.py`. Эти обёртки предварительно обрабатывают наблюдения (производят преобразования размера, цвета фрейма, взятия максимума между фреймами, пропускают часть фреймов и сводят несколько фреймов в один большой) и вознаграждения. Некоторые обёртки помогают автоматически перезапустить среду и присвоить переменной `done` значение `True` в случае смерти агента. Файл `env_batch.py` включает в себя реализацию класса `ParallelEnvBatch`, позволяющего запускать несколько сред параллельно. Для создания (инициализации) среды можно воспользоваться функцией `nature_dqn_env`. Обратите внимание, что в случае использования `PyTorch` (https://pytorch.org/) без `tensorboardX` (https://github.com/lanpa/tensorboardX) потребуется самостоятельно реализовать обёртку среды, которая будет логрировать **исходные** суммарные награды, которые *исходная* среда возвращает, и переопределить реализацию функции `nature_dqn_env`. То есть настоятельно рекомендуется применить `tensorboardX`.

Псевдокод алгоритма `Advantage Actor Critic (A2C)` приведён в **Разделе 5.2.5 (Алгоритм 20)** конспекта лекций: https://arxiv.org/pdf/2201.09746.pdf

Скрипты в данной работе используют Python версию библиотеки [OpenCV](https://pypi.org/project/opencv-python/). Для запуска сред ATARI понадобится библиотека [The Arcade Learning Environment](https://github.com/Farama-Foundation/Arcade-Learning-Environment), а также библиотека [Shimmy](https://pypi.org/project/Shimmy/). Может потребоваться установка `ffmpeg` кодека. Для Unix-based операционной системы с пакетным менеджером APT может потребоваться следующая последовательность команд:
```
sudo apt-get install -y xvfb x11-utils ffmpeg python-opengl
pip install pyglet pyvirtualdisplay opencv-python tqdm numpy moviepy gymnasium[atari]
pip install gymnasium[accept-rom-license] #run 'pip install autorom; AutoROM --accept-license' if this fails
```
Для MacOS систем рекомендуется установить репозиторий [TFM](https://github.com/serrodcal-MII/TFM/tree/main) и выполнить инструкции из [README.md](https://github.com/serrodcal-MII/TFM/blob/main/README.md).

In [1]:
rm -rf ./logs/

In [2]:
from tqdm.notebook import tqdm
import numpy as np
import os
from pathlib import Path
from IPython.display import HTML
from collections import defaultdict

import torch
import torch.optim as opt
from torch.distributions import Categorical

from gymnasium.wrappers import RecordVideo

from atari_wrappers import nature_dqn_env
from runners import EnvRunner

In [3]:
# XVFB будет запущен в случае исполнения на сервере
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

In [4]:
nenvs = 64

env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=nenvs, summaries = "Tensorboard")
                                                    # nenvs -- количество параллельно запущенных сред
                                                    # данный параметр можно варьировать для баланса
                                                    # производительность итерации/надёжность Монте-Карло оценок
                                                    # помните: при уменьшении nenvs, возможно, придётся
                                                    # увеличить количество итераций оптимизации
n_actions = env.action_space.spaces[0].n
obs, _ = env.reset()
assert obs.shape == (nenvs, 4, 84, 84)
assert obs.dtype == np.float32

A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+01

Следующим шагом будет реализация модели, которая выводит логиты для категориального распределения на действия и оценку на значения $V$-функции ценности. Рекомендуется использовать архитектуру модели, представленной в публикации в журнале [Nature](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) со следующей модификацией: вместо одного выходного слоя нужно сделать два слоя, принимающих в качестве входа выход предшествующего скрытого слоя. **Обратите внимание**, данная модель отличается от модели, предложенной в домашней работе по DQN. Рекомендуется использовать ортогональную инициализацию с параметром $\sqrt{2}$ для ядер свёрток и инициализировать смещения нулями.

In [5]:
def conv2d_size_out(size, kernel_size, stride):
    """
    общий принцип использования данной функции:
    cur_layer_img_w = conv2d_size_out(cur_layer_img_w, kernel_size, stride)
    cur_layer_img_h = conv2d_size_out(cur_layer_img_h, kernel_size, stride)
    для вычисления размерности входа полносвязного слоя
    """
    return (size - (kernel_size - 1) - 1) // stride  + 1


class Flatten(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x.reshape(x.size(0), -1)


class NatureModel(torch.nn.Module):
    
    def __init__(self, bsize, nchannels, height, width, n_actions, device):
        super().__init__()
        self.device = device
        
        self.conv1 = torch.nn.Conv2d(in_channels=nchannels, out_channels=32, kernel_size=8, stride=4)
        self.relu1 = torch.nn.ReLU()
        cur_h, cur_w = conv2d_size_out(height, 8, 4), conv2d_size_out(width, 8, 4)
        self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2)
        self.relu2 = torch.nn.ReLU()
        cur_h, cur_w = conv2d_size_out(cur_h, 4, 2), conv2d_size_out(cur_w, 4, 2)
        self.conv3 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.relu3 = torch.nn.ReLU()
        cur_h, cur_w = conv2d_size_out(cur_h, 3, 1), conv2d_size_out(cur_w, 3, 1)
        self.flatten = Flatten()
        self.fc1 = torch.nn.Linear(in_features=64 * cur_h * cur_w, out_features=512)
        self.relu4 = torch.nn.ReLU()
        
        self.fc_logits = torch.nn.Linear(in_features=512, out_features=n_actions)
        self.fc_values = torch.nn.Linear(in_features=512, out_features=1)
        
        self._reset_parameters()
    
    def _reset_parameters(self):
        torch.nn.init.orthogonal_(self.conv1.weight.data, gain=np.sqrt(2.))
        self.conv1.bias.data.fill_(.0)
        
        torch.nn.init.orthogonal_(self.conv2.weight.data, gain=np.sqrt(2.))
        self.conv2.bias.data.fill_(.0)
        
        torch.nn.init.orthogonal_(self.conv3.weight.data, gain=np.sqrt(2.))
        self.conv3.bias.data.fill_(.0)
        
        torch.nn.init.orthogonal_(self.fc1.weight.data, gain=np.sqrt(2.))
        self.fc1.bias.data.fill_(.0)
        
        torch.nn.init.orthogonal_(self.fc_logits.weight.data, gain=np.sqrt(2.))
        self.fc_logits.bias.data.fill_(.0)
        
        torch.nn.init.orthogonal_(self.fc_values.weight.data, gain=np.sqrt(2.))
        self.fc_values.bias.data.fill_(.0)
    
    def forward(self, state):
        x = torch.tensor(np.array(state), device=self.device, dtype=torch.float)
        
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu4(x)
        
        logits = self.fc_logits(x)
        value = self.fc_values(x)
        
        x.cpu()
        del x
        
        return logits, value

Вам также потребуется определить и использовать политику, которая будет использовать модель выше. В то время как модель вычисляет логиты для всех действий сразу и оценку функции ценности, политика будет сэмплировать действия, а также будет вычислять их логарифм правдоподобия. Метод `Policy.act` должен возвращать словарь всех массивов, требуемых для взаимодействия со средой и обучения модели. Обратите внимание, что действия должны быть формата `Numpy.ndarray`, в то время как другие тензоры должны быть в формате, определяемом библиотекой глубокого обучения (`Torch.tensor`, например).

In [6]:
class Policy:
    def __init__(self, model):
        self.model = model

    def act(self, inputs):
        #<Реализуйте политику посредством прямого прохода модели, сэмплирования действий и вычисления их логарифмов правдоподобия>
        # Должен возвращать dict с ключами ['actions', 'logits', 'log_probs', 'values'].
        logits, values = self.model(inputs)
        values = values.reshape(-1)
        dist = torch.distributions.Categorical(logits = logits)
        actions = dist.sample()
        log_probs = dist.log_prob(actions)
        actions = actions.cpu().numpy()
        entropy = dist.entropy()
        return {'actions': actions, 'logits': logits, 'log_probs': log_probs, 'values': values, 'entropy': entropy} 
    def reset(self):
        pass

Далее требуется передать среду и политику в исполнитель `EnvRunner`, который собирает частичные траектории из среды. Класс `EnvRunner` уже реализован за Вас.

Данный исполнитель взаимодействует со средой заданное количество шагов и возвращает словарь, содержащий ключи:

* 'observations' 
* 'rewards' 
* 'resets'
* 'actions'
* и другие ключи, определённые в `Policy`

по каждому из этих ключей содержится Python `list` соответствующих результатов взаимодействий со средой указанной длины $T$ &mdash; размера частичной траектории.

Для того чтобы обучить часть модели, которая предсказывает ценности состояний, требуется вычислить целевые значения ценностей. В инстанцию класса `EnvRunner` можно подать при создании по аргументу `transforms` список вызываемых объектов ("функций"), которые последовательно будут применяться к частичным траекториям после сбора самих траекторий. Следовательно, требуется реализовать и использовать вызываемый (с определённым методом `__call__`) класс `ComputeValueTargets`. Формула для вычисления целевых значений ценности простая:

$$
\hat v(s_t) = \sum_{t'=0}^{T - 1}\gamma^{t'}r_{t+t'} + \gamma^T \hat{v}(s_{t+T}),
$$

Однако, не забудьте в реализации использовать `trajectory['resets']` флаги для проверки того, следует ли добавить целевые значения ценности на следующем шаге при вычислении целевых значений ценности на текущем шаге. У вас также имеется доступ к `trajectory['state']['latest_observation']` для получения последнего наблюдения в частичной траектории &mdash; $s_{t+T+1}$.

In [7]:
class ComputeValueTargets:
    def __init__(self, gamma=0.99, device=torch.device('cuda:0')):
        self.gamma = gamma
        self.device = device

    def __call__(self, trajectory):
        # Данный метод должен модифицировать траекторию trajectory на месте через добавление
        # итеративно заполненного списка по ключу 'value_targets'.
        rewards = torch.tensor(np.stack(trajectory['rewards']), device = self.device, dtype=torch.float32)
        value_targets = torch.tensor(np.stack(trajectory['rewards']), device = self.device, dtype=torch.float32)
        resets = torch.tensor(np.stack(trajectory['resets']), device = self.device, dtype=torch.float32)

        values = torch.stack(trajectory['values'])
        
        value_targets[-1] = values[-1] * (1 - resets[-1]) +  rewards[-1] * resets[-1]
        for t in range(trajectory['state']['env_steps'] - 2, -1, -1):
            value_targets[t] = value_targets[t] + (value_targets[t + 1] * self.gamma) * (1 - resets[t])
        trajectory['value_targets'] = value_targets
        

После вычисления целевых значений ценности требуется преобразовать списки результатов взаимодействия со средой в тензоры с первой компонентой размерности `batch_size`, равной призведению `T * nenvs`, то есть требуется свести в одну компоненту первые две компоненты размерности.

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [9]:
class MergeTimeBatch:
    """
    Сращивает первые две оси, обычно отвечающие за время и за инстанцию среды, соответственно.
    """
    def __call__(self, trajectory):
        # Модификация траектории на месте. 
        device = trajectory['value_targets'].device
        trajectory['rewards'] = torch.tensor(np.stack(trajectory['rewards']), device = device, dtype=torch.float32).reshape(-1)
        trajectory['resets'] = torch.tensor(np.stack(trajectory['resets']), device = device, dtype=torch.float32).reshape(-1)
        trajectory['values'] = torch.stack(trajectory['values']).to(device).reshape(-1)
        trajectory['entropy'] = torch.stack(trajectory['entropy']).to(device).reshape(-1)
        trajectory['value_targets'] = trajectory['value_targets'].reshape(-1)
        trajectory['log_probs'] = torch.stack(trajectory['log_probs']).to(device).reshape(-1)
        

In [10]:
seed = 617 # на своё усмотрение можно выбрать другой seed
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = True
nsteps=5
model = NatureModel(*obs.shape, n_actions, device)
model.to(device)
policy = Policy(model)
torch.save(model.state_dict(), 'agent_actor_model_init.pt')
#model.load_state_dict(torch.load('agent_actor_model_init.pt'))
runner = EnvRunner(env, policy, nsteps=5, # nsteps -- длина частичной траектории
                                          # уменьшение nsteps может привести к вынужденному увеличению
                                          # количества итераций оптимизации
                   transforms=[ComputeValueTargets(device=device),
                               MergeTimeBatch()])

Настало время реализовать сам алгоритм Advantage-Actor Critic (A2C). Его псевдокод можно посмотреть в [конспектах лекций (Раздел 5.2.5)](https://arxiv.org/pdf/2201.09746.pdf), в публикции [Mnih et al. 2016](https://arxiv.org/abs/1602.01783) и в [лекции](https://www.youtube.com/watch?v=Tol_jw5hWnI&list=PLkFD6_40KJIxJMR-j5A1mkxK26gh_qg37&index=20) Сергея Левина.

In [11]:
class A2C:
    def __init__(self,
               policy,
               optimizer,
               value_loss_coef=.25,
               entropy_coef=1e-2,
               max_grad_norm=.5):
        self.policy = policy
        self.optimizer = optimizer
        self.value_loss_coef = value_loss_coef
        self.entropy_coef = entropy_coef
        self.max_grad_norm = max_grad_norm

    def policy_loss(self, trajectory):
        # Тут нужно вычислить Advantages. 
        #<TODO: реализовать>
        advantages = trajectory['value_targets'] -  trajectory['values']
        advantages = advantages.detach()
        return -(trajectory['log_probs'] * advantages).mean()
    def value_loss(self, trajectory):
        #<TODO: реализовать>
        return torch.nn.MSELoss()(trajectory['values'], trajectory['value_targets'].detach())

    def loss(self, trajectory):
        entropy = trajectory['entropy'].mean()
        
        return self.value_loss_coef * self.value_loss(trajectory) + self.policy_loss(trajectory) - (self.entropy_coef * entropy)

    def step(self, trajectory):
        self.optimizer.zero_grad()
        loss = self.loss(trajectory)
        loss.backward()
        norm = torch.nn.utils.clip_grad_norm_(model.parameters(), self.max_grad_norm)
        self.optimizer.step()
        return loss, norm

Теперь можно непосредственно обучить Вашу модель. С разумно подобранными гиперпараметрами обучение на одной GTX1080 на протяжении 10 миллионов шагов суммарно со всех батчированных сред (что переводится примерно в 5 часов работы) должно быть возможно достигнуть *среднюю исходную награду за 100 последних эпизодов* (значение переменной в `Tensorboard` по ключу `reward_mean_100`, усреднение берётся по 100 последним эпизодам в каждой среде в батче) **не меньше 600**. Это и будет считаться успешным результатом работы алгоритма `A2C`.

**Внимание!** При *коректной* имплементации алгоритма *обоснованное* преодоление порога **400** по `reward_mean_100` *транслируется в* **8 баллов за задание**, *обоснованное* преодоление порога **600** по `reward_mean_100` на *корректно* написанном алгоритме обучения `A2C` *транслируется в* **10 баллов за задание**.

Вам так же, по возможности, рекомендуется отобразить данную величину относительно `runner.step_var` &mdash; количества взаимодействий со всеми средами. Также очень рекомендуется предоставить графики следующих показателей (полезно для отладки кода):
* [Коэффициент детерминации](https://en.wikipedia.org/wiki/Coefficient_of_determination) между целевыми значениями ценности и их предсказаниями
* Энтропия политики $\pi$
* Функция потерь ценности (Value loss)
* Функция потерь политики (Policy loss)
* Целевые значения ценности (Value targets)
* Предсказания значений ценности (Value predictions)
* Норма градиента
* Advantages
* Общая функция потерь (A2C loss)

В качестве оптимизатора рекомендуется взять метод [RMSProp](https://pytorch.org/docs/stable/generated/torch.optim.RMSprop.html) с [линейным убыванием шага](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.LambdaLR.html), начиная с 7e-4 до 0, константой сглаживания (alpha в PyTorch и decay в TensorFlow), равной 0.99 и epsilon, равным 1e-5.

Запуск Tensorboard

In [12]:
from tensorboardX import SummaryWriter

In [13]:
%load_ext tensorboard
%tensorboard --logdir logs

In [14]:
import torcheval.metrics
writer = SummaryWriter('./logs/run-1')
lr = 7e-4
alpha=0.99
eps=1e-05
total_steps = int(80e6)
eval_freq = 100
optimizer = torch.optim.RMSprop(model.parameters(), lr = lr, alpha=alpha, eps=eps)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda t: (total_steps - t) / total_steps)
a2c = A2C(policy, optimizer, value_loss_coef=.25, entropy_coef=1e-2, max_grad_norm=.5)
r2metric =  torcheval.metrics.R2Score()
batch_size = nenvs*nsteps
for step in tqdm(range(total_steps//batch_size+ 1)):
    trajectory = runner.get_next()
    loss, norm = a2c.step(trajectory)
    scheduler.step()
    if(step%eval_freq == 0):
        writer.add_scalar('Eval/Entropy', trajectory['entropy'].mean(), step)
        writer.add_scalar('Eval/value_loss', a2c.value_loss(trajectory), step)
        writer.add_scalar('Eval/policy_loss', a2c.policy_loss(trajectory), step)
        writer.add_scalar('Eval/loss', a2c.loss(trajectory), step)
        r2metric.update(trajectory['values'].cpu(), trajectory['value_targets'].detach().cpu())
        writer.add_scalar('Eval/R2Metric', r2metric.compute(), step)
        writer.add_scalar('Eval/gradnorm', norm, step)
        torch.save(model.state_dict(), 'agent_actor_model.pt')

    

  0%|          | 0/250001 [00:00<?, ?it/s]

In [15]:
writer.close()

In [16]:
test_env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=None, clip_reward=False, summaries=False)

A.L.E: Arcade Learning Environment (version 0.8.1+01d3e3a)
[Powered by Stella]


In [17]:
n_lives = 3

def evaluate(env, agent, n_games=1, greedy=False, t_max=10000):
    """
    Играем n_games игр до конца.
    В случае жадной политики, выбираем действия как argmax(qvalues).
    Возвращаем среднюю награду.
    """
    rewards = []
    for _ in range(n_games):
        s, _ = env.reset()
        reward = 0
        for _ in range(t_max):
            output = agent.act([s])
            action = output['logits'].argmax(dim=-1).item() if greedy else output['actions'][0]
            s, r, terminated, truncated, _ = env.step(action)
            reward += r
            if terminated or truncated:
                break

        rewards.append(reward)
    return np.mean(rewards)

In [18]:
# запись эпизодов
with RecordVideo(
    env=test_env,
    video_folder="./videos/False",
    episode_trigger=lambda episode_number: True
) as env_monitor:
    sessions = [evaluate(env_monitor, policy, n_games=n_lives, greedy=False) for _ in range(1)]

/home/sasha/anaconda3/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:364: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Moviepy - Building video /home/sasha/RL4/videos/False/rl-video-episode-0.mp4.
Moviepy - Writing video /home/sasha/RL4/videos/False/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/sasha/RL4/videos/False/rl-video-episode-0.mp4
Moviepy - Building video /home/sasha/RL4/videos/False/rl-video-episode-1.mp4.
Moviepy - Writing video /home/sasha/RL4/videos/False/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /home/sasha/RL4/videos/False/rl-video-episode-1.mp4


Moviepy - Building video /home/sasha/RL4/videos/False/rl-video-episode-2.mp4.
Moviepy - Writing video /home/sasha/RL4/videos/False/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /home/sasha/RL4/videos/False/rl-video-episode-2.mp4


In [19]:
video_paths = sorted([s for s in Path('videos', 'False').iterdir() if s.suffix == '.mp4'])
video_path = video_paths[1]
data_url = str(video_path)

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(data_url))

In [20]:
# запись эпизодов
with RecordVideo(
    env=test_env,
    video_folder="./videos/True",
    episode_trigger=lambda episode_number: True
) as env_monitor:
    sessions = [evaluate(env_monitor, policy, n_games=n_lives, greedy=True) for _ in range(1)]

Moviepy - Building video /home/sasha/RL4/videos/True/rl-video-episode-0.mp4.
Moviepy - Writing video /home/sasha/RL4/videos/True/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/sasha/RL4/videos/True/rl-video-episode-0.mp4
Moviepy - Building video /home/sasha/RL4/videos/True/rl-video-episode-1.mp4.
Moviepy - Writing video /home/sasha/RL4/videos/True/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /home/sasha/RL4/videos/True/rl-video-episode-1.mp4


Moviepy - Building video /home/sasha/RL4/videos/True/rl-video-episode-2.mp4.
Moviepy - Writing video /home/sasha/RL4/videos/True/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /home/sasha/RL4/videos/True/rl-video-episode-2.mp4


In [21]:
video_paths = sorted([s for s in Path('videos', 'True').iterdir() if s.suffix == '.mp4'])
video_path = video_paths[1]
data_url = str(video_path)

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(data_url))

# Информация об обучении

Прикрепление скриншотов графиков обучения модели в `Tensorboard` ниже является обязательным. Для доступа к `Tensorboard` запустите из командной строки в одной директории с данным ноутбуком следующую команду:
```
tensorboard --logdir logs --port 6006
```
В результате вывод в командную строку укажет, по какому адресу можно подсоединиться к инстанции `Tensorboard`, например, по адресу `http://localhost:6006/`. Оттуда можно и сделать скриншоты, демонстрирующие результаты обучения модели. Сами скриншоты с именем файла `image_name_x.png` для удобства лучше сохранить в директорию `./img`, откуда можно легко их прикреплять в `Markdown-клетках` ниже по команде со следующей конструкцией:
```
<img src=./img/image_name_x.png width=640>
```
Тут также требуется подписать изображения и дать небольшой комментарий по каждому скриншоту, что на нём описано.

**Внимание!** В случае перезапуска процедуры обучения модели рекомендуется удалить директорию `./logs` вместе с её содержимым перед непосредственным перезапуском, чтобы не испортить отображающиеся графики в `Tensorboard`.

**Совет.** При работе в Google Colab можно просто скачать директорию `./logs` и уже локально запустить `Tensorboard` для снятия скриншотов. Также можно обученного агента сохранить, скачать и локально на cpu запустить для записи роликов (для этого понадобится самостоятельно прописать код сохранения и загрузки модели в ноутбук из [файла](https://pytorch.org/tutorials/beginner/saving_loading_models.html)).

**Внимание!** Посылку для сдачи задания требуется оформить в виде `.zip` архива, в котором будут *данный ноутбук*, использованные для его работы *скрипты*, *директории* `./videos`, `./logs` и `./img` с содержимым. Только так и не иначе!

__Вставьте в данную ячейку свой ответ, подкреплённый скриншотами__

<img src=./img/episode_legth_max_reward.png width=640>

Длина эпизода коррелирует с оценкой reward_mean_100. Как видим максимальная награда грастет и на порядок превышает среднюю оценку, что говорит о широком разнообразии просматриваемых решений, при данной политики, при это модел ьпродолжает обучаться

<img src=./img/min_reward_reward_mean_100.png width=640>

Минимальная награда не меняется особо и находится на около 0 уровне, что говорит о том, что политика не вырожддается и мы рассматриваем даже самые непродуктивные случаи, в угоду возможностям исследования. Средняя награда говорит о том что найден примерный балн между исследванием и наградами. Модель имеет достаточно широкую свободу для исследований, при этом не забывая уделять время продуктивным эпизодам, чт ов среднем дает нелохой темп обучения.

<img src=./img/Entropy_R2Metric.png width=640>

Энтропия выступет в качестве меры случайности policy-net, в целом постепенно находя оптимальную политику ее случайность уменьшается, но пока она далека от вырожденного детерминированного случая. 

R2-Metric изначально у меня получалась около 0, что говорило об ошибке в коде при подсчете target_value. В идеале target_value и value должны оставаться скоррелированны, т.к. наиболее вероятный исход, что предыдуще занчение value совпадает с текущим (т.к. награда у нас всего лишь 1).

<img src=./img/gradnorm_loss.png width=640>

gradnorm в целом находится меньше 0.5, но под конец градиент пришлось клиповать. 

<img src=./img/policy_loss_value_loss.png width=640>

Для нахождения баланса между value_loss и policy_loss нужно чтобы каждый из loss'ов коррелировал с loss, т.е. чтобы они давалаи примено одинаковый вклад в него. Таким образом был подобран весовой коэффициент